In [1]:
import qlib
from qlib.data import D

In [2]:
qlib.init(provider_uri='~/.qlib/qlib_data/cn_data')

[33924:MainThread](2025-12-25 10:41:40,404) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[33924:MainThread](2025-12-25 10:41:40,662) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[33924:MainThread](2025-12-25 10:41:40,663) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/iamwu/.qlib/qlib_data/cn_data')}


In [3]:
from qlib.data import D

df = D.features(
    ["SH601216"],
    ["$open","$high","$low","$close","$factor"],
    start_time="2020-05-01",
    end_time="2020-05-31",
    freq="day"
)
print(df.head())

                          $open     $high      $low    $close   $factor
instrument datetime                                                    
SH601216   2020-05-06  1.400656  1.430144  1.385912  1.420314  0.491458
           2020-05-07  1.420314  1.420314  1.400656  1.405571  0.491458
           2020-05-08  1.410485  1.425229  1.405571  1.410485  0.491458
           2020-05-11  1.415400  1.420314  1.405571  1.410485  0.491458
           2020-05-12  1.410485  1.420314  1.405571  1.410485  0.491458


In [4]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x = df.index.get_level_values("datetime"),
                                     open=df["$open"],high=df["$high"],low=df["$low"],close=df["$close"])])
fig.show()

In [5]:
# 价格调成，解决送股，分红导致的价格跳跃问题
fig = go.Figure(data=[go.Candlestick(x = df.index.get_level_values("datetime"),
                                     open=df["$open"]/df["$factor"],high=df["$high"]/df["$factor"],low=df["$low"]/df["$factor"],close=df["$close"]/df["$factor"])])
fig.show()

In [6]:
from qlib.contrib.strategy.signal_strategy import BaseSignalStrategy
from qlib.backtest.decision import Order, TradeDecisionWO

class DoubleMAStrategy(BaseSignalStrategy):
    def __init__(self, instruments, short_window, long_window):
        self.instruments = instruments
        self.short_window = short_window
        self.long_window = long_window


    def generate_trade_decision(self, execute_result=None):
        trade_step = self.trade_calendar.get_trade_step()
        trade_start_time, trade_end_time = self.trade_calendar.get_step_time(trade_step)
        pred_start_time, pred_end_time = self.trade_calendar.get_step_time(trade_step, shift=1)
        pred_score = self.signal.get_signal(start_time=pred_start_time, end_time=pred_end_time)
        data = D.features(
                instruments=self.instruments,
                fields=["$close"],
                start_time=trade_start_time,
                end_time=trade_end_time,
                freq="day"
                ).dropna()

        short_ma = data["$close"].rolling(window=self.short_window).mean()
        long_ma = data["$close"].rolling(window=self.long_window).mean()
        
        if short_ma.iloc[-1] > long_ma.iloc[-1]:
            return TradeDecisionWO([Order(stock_id=self.instruments, direction=Order.BUY)],self)
        elif short_ma.iloc[-1] < long_ma.iloc[-1]:
            return TradeDecisionWO([Order(stock_id=self.instruments, direction=Order.SELL,start_time=trade_start_time,end_time=trade_end_time,amount=1)],self)
        return TradeDecisionWO([],self)
    

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [7]:
from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.strategy import TopkDropoutStrategy 
strategy_config = {
   "instruments": ["SH600519"], # 股票代码，如贵州茅台
   "short_window": 5,
   "long_window": 20,
}
strategy_obj = DoubleMAStrategy(**strategy_config)
report, positions = backtest_daily(
   start_time="2020-01-01",
   end_time="2021-01-01",
   strategy=strategy_obj,
)
# print(report)

[33924:MainThread](2025-12-25 10:41:43,356) WARNING - qlib.data - [data.py:665] - load calendar error: freq=day, future=True; return current calendar!
[33924:MainThread](2025-12-25 10:41:43,356) WARNING - qlib.data - [data.py:668] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[33924:MainThread](2025-12-25 10:41:43,363) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x0000027A2476E940>
[33924:MainThread](2025-12-25 10:41:43,368) INFO - qlib.backtest caller - [__init__.py:93] - Create new exchange
[33924:MainThread](2025-12-25 10:42:07,225) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[33924:MainThread](2025-12-25 10:42:07,226) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[33924:MainThread](2025-12-25 10:42:07,230) WAR

backtest loop:   0%|          | 0/180 [00:00<?, ?it/s]

AttributeError: 'DoubleMAStrategy' object has no attribute 'signal'

In [ ]:
strategy_obj = TopkDropoutStrategy(topk=50,n_drop=5)

report, positions = backtest_daily(
   start_time="2020-01-01",
   end_time="2021-01-01",
   strategy=strategy_obj,
)

NotImplementedError: This type of signal is not supported